In [1]:
from fbs.bpfbs import *

In [2]:
import pickle

fn_binary = 'df_count_slices.pkl'
with open(fn_binary, 'rb') as f:
    df = pickle.load(f)

df.head()

,size,count,relfreq,pdb_code,b
0,25,31,0.000065,2vda,1011110111101111011110111
1,25,29,0.000060,2lfs,1011110111101111011110111
2,25,23,0.000048,2kz1,1011110111101111011110111
3,25,21,0.000044,6h5h,1011110111101111011110111
4,25,20,0.000042,2ld4,1011110111101111011110111


In [3]:
seed = 42
pdb_train, pdb_test = train_test_split(df['pdb_code'].unique(), train_size=0.8, random_state=seed)
df_train = df[df['pdb_code'].isin(pdb_train)]
states = list(df_train['b'])
p = list(df_train['relfreq'])

print('states: ', states[:5])
print('p: ', p[:5])

states:  ['1011110111101111011110111', '1011110111101111011110111', '1011110111101111011110111', '1011110111101111011110111', '1011110111101111011110111']
p:  [6.0383330036542327e-05, 4.789022727036116e-05, 4.164367588727057e-05, 4.164367588727057e-05, 3.956149209290704e-05]


In [4]:
import numpy as np
from tqdm import tqdm

wd_binary = 'binary'
wd_original_sol = 'original_sol'

def generate_tests(pdb_test):    
    M = {'fn':[], 'order': [], 'parents': [], 'xsol':[]}
    
    pdb_test = set(pdb_test)
    for fn_binary in tqdm(os.listdir(wd_binary)):
        pdb_code = fn_binary.split('_')[0]
        if not pdb_code in pdb_test:
            continue

        fn_binary = os.path.join(wd_binary, fn_binary)
        df = pd.read_csv(fn_binary)
        
        df.fillna(-1, inplace=True)

        # convert from b (binary) to s (string)
        M['fn'].append(fn_binary)
        M['order'].append(df['order'].values)
        parents = zip(df['N_1'].astype(int).values, df['N_2'].astype(int).values, df['N_3'].astype(int).values)
        # convert parents to list
        M['parents'].append([list(p) for p in parents])

        fn_xsol = os.path.basename(fn_binary).replace('_binary.csv', '.sol')
        fn_xsol = os.path.join(wd_original_sol, fn_xsol)

        M['xsol'].append(np.loadtxt(fn_xsol))

    M = pd.DataFrame(M)

    return M

M = generate_tests(pdb_test)
M 

100%|██████████| 73649/73649 [08:00<00:00, 153.20it/s]


,fn,order,parents,xsol
0,binary\1a03_model1_chainA_segment0_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[-12.449, 7.053, 5.066], [-11.098, 6.511, 5.1..."
1,binary\1a03_model1_chainA_segment1_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[-11.299, -7.574, 2.87], [-11.467, -8.365, 1...."
2,binary\1a03_model1_chainA_segment2_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[-10.069, -18.068, 5.889], [-9.382, -16.777, ..."
3,binary\1a03_model1_chainA_segment3_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[-7.242, -12.744, -21.486], [-7.716, -11.455,..."
4,binary\1a03_model1_chainA_segment4_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[1.536, -0.32, -6.37], [1.878, -0.516, -7.772..."
...,...,...,...,...
14746,binary\8wls_model1_chainA_segment4_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[15.863, -5.167, 6.072], [14.924, -4.241, 6.7..."
14747,binary\8wls_model1_chainA_segment5_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[-9.888, -4.909, 3.205], [-8.464, -4.721, 3.5..."
14748,binary\8wls_model1_chainA_segment6_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[2.196, -2.452, -6.283], [2.824, -1.142, -6.4..."
14749,binary\8wls_model1_chainA_segment7_binary.csv,"[0, 1, 3, 2, 7, 4, 5, 8, 6, 12, 9, 10, 13, 11,...","[[-1, -1, -1], [-1, -1, -1], [-1, -1, -1], [0,...","[[-8.246, -17.427, 7.782], [-9.374, -16.486, 7..."


In [5]:
import plotly.express as px

M['size'] = M['xsol'].apply(lambda x: x.shape[0])
df_ = M.groupby('size').count().reset_index()

fig = px.bar(df_, x='size', y='fn')
fig.show()

In [17]:
import pandas as pd
import numpy as np

for _, row in tqdm(list(M.iterrows())):
    order = row['order']
    
    n = len(order)
    row_parents = row['parents']
    parents = [[] for i in range(n)]
    for i in range(n):
        parents[i] = row_parents[i]
    
    
    # The atoms follow the following ddgp order: N^1, CA^1, HA^1, ..., C^{j-1}, H^j, N^j, CA^j, HA^j, ..., C^nres
    d = {}
    xsol = row['xsol']
    # exact distances between the first three atoms
    for i in range(3):
        d[i] = {}
        for j in range(i-1, 0, -1):
            d[i][j] = norm(xsol[order[i]] - xsol[order[j]])
    
    H = []
    # exact distances associated with the atoms of each block ( C^{j-1}, H^j, N^j, CA^j, HA^j ), with j in {2,n}
    for i in range(3, n-1, 5):
        # i: C^{j-1}
        d[i] = {}
        d[i][i-1] = norm(xsol[order[i]] - xsol[order[i-1]])
        d[i][i-2] = norm(xsol[order[i]] - xsol[order[i-2]])
        d[i][i-3] = norm(xsol[order[i]] - xsol[order[i-3]])
        
        # i+1: H^j
        d[i+1] = {}
        d[i+1][i] = norm(xsol[order[i+1]] - xsol[order[i]])
        d[i+1][i-1] = norm(xsol[order[i+1]] - xsol[order[i-1]])
        d[i+1][i-2] = norm(xsol[order[i+1]] - xsol[order[i-2]])
        H.append(i+1)

        # i+2: N^j
        d[i+2] = {}
        d[i+2][i+1] = norm(xsol[order[i+2]] - xsol[order[i+1]])
        d[i+2][i] = norm(xsol[order[i+2]] - xsol[order[i]])
        d[i+2][i-2] = norm(xsol[order[i+2]] - xsol[order[i-2]])

        # i+3: CA^j
        d[i+3] = {}
        d[i+3][i+2] = norm(xsol[order[i+3]] - xsol[order[i+2]])
        d[i+3][i+1] = norm(xsol[order[i+3]] - xsol[order[i+1]])
        d[i+3][i] = norm(xsol[order[i+3]] - xsol[order[i]])
        d[i+3][i-2] = norm(xsol[order[i+3]] - xsol[order[i-2]])

        # i+4: HA^j
        d[i+4] = {}
        d[i+4][i+3] = norm(xsol[order[i+4]] - xsol[order[i+3]])
        d[i+4][i+2] = norm(xsol[order[i+4]] - xsol[order[i+2]])
        d[i+4][i+1] = norm(xsol[order[i+4]] - xsol[order[i+1]])
        H.append(i+4)
    
    # distances associated with the atom C^nres
    i = i+5
    d[i] = {}
    d[i][i-1] = norm(xsol[order[i]] - xsol[order[i-1]])
    d[i][i-2] = norm(xsol[order[i]] - xsol[order[i-2]])
    d[i][i-3] = norm(xsol[order[i]] - xsol[order[i-3]])

    # distances between hydrogen atoms that are not the adjacent and that are at most 5 angstroms apart
    for i in range(len(H)):
        for j in range(i-2):
            # (i+1: H^j) and (i+4: HA^j)
            dij = norm(xsol[order[H[i]]] - xsol[order[H[j]]])
            if dij <= 5:
                d[i][j] = dij

    # symmetrize
    for i in range(n):
        for j in d[i].keys():
            d[j][i] = d[i][j]
    
    D = DDGP(d, parents)
    arroz = 1


        

    

 41%|████      | 6038/14751 [04:41<06:45, 21.48it/s]   


KeyboardInterrupt: 

In [10]:
import pickle
import pandas as pd

wd_original_sol = 'original_sol'

all_solutions = {}
for fn_sol in tqdm(os.listdir(wd_original_sol)):
    fn_sol = os.path.join(wd_original_sol, fn_sol)
    this_df = pd.read_csv(fn_sol)
    all_solutions[fn_sol] = this_df

with open('pickled_original_sol', 'wb') as f:
    pickle.dump(all_solutions, f)

100%|██████████| 73649/73649 [21:08<00:00, 58.06it/s] 


In [9]:
import pickle
import pandas as pd

wd_binary = 'binary'
wd_original_sol = 'original_sol'

all_binary_files = {}
for fn_binary in tqdm(os.listdir(wd_binary)):
    fn_binary = os.path.join(wd_binary, fn_binary)
    this_df = pd.read_csv(fn_binary)
    all_binary_files[fn_binary] = this_df

with open('pickled_binary', 'wb') as f:
    pickle.dump(all_binary_files, f)
    

100%|██████████| 73649/73649 [03:00<00:00, 408.44it/s]
